In [1]:
import QDMPy.data_loading as data_loading
import QDMPy.fit_interface as fit_interface
import QDMPy.fit_plots as fit_plots

import matplotlib
import matplotlib.pyplot as plt

In [2]:
# '%matplotlib widget' allows interactive plots in jupyterlab
# use ''%matplotlib inline' to allow export to pdf
%matplotlib widget

# Load the data in

In [10]:
options_dict = {
    "base_dir": "C:\\src\\QDMPY_test_image\\",
    "filepath": "ODMR - Pulsed with IR_83",
    
    "additional_bins": 1,

    "system_name": "Zyla",

    "ROI": "Rectangle", 
    "ROI_start": [0, 0],
    "ROI_end": [500,500],
    
    "AOI_1_start": [200, 300],
    "AOI_1_end": [240, 340], 
    "AOI_1_start": [200, 200],
    "AOI_2_end": [240, 240], 
    "AOI_1_start": [300, 200],
    "AOI_3_end": [320, 240], 

    "normalisation": "div", # fix this for consistent fitting -> get to 0->1?
    
    "fit_backend": "gpufit",
    "fit_backend_comparison": ["scipy", "gpufit"],
    "fit_pixels": True, 
    "force_fit": True, 
    "auto_match_prev_ROI_options": False,
    
    "use_ROI_avg_fit_res_for_all_pixels": True,
    
    "scipy_sub_threads": 2, 

    "fit_functions":  {"linear": 1, "lorentzian": 8},

    "pos_guess": [2610, 2708, 2805, 2879, 2953, 3020, 3092, 3145],
    "pos_range": 25,

    "amp_guess": -0.0015,
    "amp_bounds": [-0.0100, -0.0003],

    "fwhm_guess": 9,
    "fwhm_bounds": [5, 20],

    "c_guess": 1,
    "c_bounds": [0.9995, 1.0005],

    "m_guess": 0,
    "m_bounds": [-1e-6, 1e-6],
    
    "scramble_pixels": True,
    
    "gpufit_tolerance": 1e-12,
    "gpufit_max_iterations": 25,
    "gpufit_estimator_id": "LSE"
}

In [11]:
# QDMPy_path = data_loading.DIR_PATH
# options = data_loading.load_options(path=QDMPy_path / "options/test_options_odmr.json", check_for_prev_result=True)
options = data_loading.load_options(options_dict=options_dict, check_for_prev_result=True)
fit_plots.set_mpl_rcparams(options)

In [12]:
raw_data, prelim_sweep_list = data_loading.load_raw_and_sweep(options)

In [13]:
PL_image, PL_image_ROI, sig, ref, sig_norm, single_pixel_pl, sweep_list = data_loading.reshape_dataset(
        options, raw_data, prelim_sweep_list
    )

C:\src\QDMPy\QDMPy\data_loading.py:573: UserWarning: ROI Rectangle too big in x, cropping to image.
  warnings.warn("ROI Rectangle too big in x, cropping to image.")
C:\src\QDMPy\QDMPy\data_loading.py:579: UserWarning: ROI Rectangle too big in y, cropping to image.
  warnings.warn("ROI Rectangle too big in y, cropping to image.")


In [14]:
fig1 = fit_plots.plot_ROI_PL_image(options, PL_image)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
AOIs = data_loading.define_AOIs(options)

In [16]:
fig2 = fit_plots.plot_AOI_PL_images(options, PL_image_ROI, AOIs)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
aoi_spectra_plot = fit_plots.plot_AOI_spectra(options, AOIs, sig, ref, sweep_list)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit AOI, ROI averages, single pixel

In [ ]:
fit_model = fit_interface.define_fit_model(options)

In [ ]:
backend_ROI_results_lst = fit_interface.fit_ROI_avg(options, sig_norm, sweep_list, fit_model)

In [ ]:
fig3 = fit_plots.plot_ROI_avg_fits(options, backend_ROI_results_lst)

In [ ]:
fit_result_collection_lst = fit_interface.fit_AOIs(options, sig_norm, single_pixel_pl, sweep_list, fit_model, AOIs, backend_ROI_results_lst)

In [ ]:
AOI_spectra_fit_fig = fit_plots.plot_AOI_spectra_fit(options, sig, ref, sweep_list, AOIs, fit_result_collection_lst, fit_model)

# Fit pixels

In [ ]:
if (options["force_fit"] or not options["found_prev_result"]) and options["fit_pixels"]:
    pixel_fit_params = fit_interface.fit_pixels(options, sig_norm, sweep_list, fit_model, backend_ROI_results_lst[0])
elif options["found_prev_result"]:
    pixel_fit_params = fit_interface.load_prev_fit_results(options)
else:
    pixel_fit_params = None  # not fitting pixels, this stops plotting (e.g. via plot_param_images) from erroring

# Plot (fit) parameters

In [ ]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "c")

In [ ]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "m")

In [ ]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "pos")

In [ ]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "amp")

In [ ]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "fwhm")

In [ ]:
data_loading.save_options(options)

In [ ]:
options